In [39]:
from gensim.models import Word2Vec
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
# Download the punkt tokenizer for sentence splitting

train = pd.read_csv( "/Users/aramamurthy/Downloads/Github/ANLP/Assignment9/labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3,encoding="utf8" )
test = pd.read_csv( "/Users/aramamurthy/Downloads/Github/ANLP/Assignment9/testData.tsv", header=0, 
                   delimiter="\t", quoting=3,encoding="utf8" )
unlabeled_train = pd.read_csv( "/Users/aramamurthy/Downloads/Github/ANLP/Assignment9/unlabeledTrainData.tsv", 
                              header=0, delimiter="\t", quoting=3,encoding="utf8" )


In [2]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [3]:
import nltk
nltk.download()

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

In [41]:
len(sentences)

267979

In [42]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

Training model...


In [51]:
model.doesnt_match("man woman child cattle".split())

'cattle'

In [73]:
model.doesnt_match("donkey horse cow men".split())

'men'

In [72]:
model.most_similar("ego")

[(u'ambition', 0.6574293375015259),
 (u'obsessive', 0.6557430028915405),
 (u'destiny', 0.6487337946891785),
 (u'alter', 0.6384143829345703),
 (u'soul', 0.6279738545417786),
 (u'mate', 0.6243647336959839),
 (u'authority', 0.6207817196846008),
 (u'unstable', 0.6151727437973022),
 (u'impending', 0.6073411703109741),
 (u'innocence', 0.6046621799468994)]

In [54]:
model.most_similar("awkward")

[(u'unconvincing', 0.7432976365089417),
 (u'stilted', 0.7388715744018555),
 (u'uncomfortable', 0.7375034093856812),
 (u'artificial', 0.7350825071334839),
 (u'irritating', 0.7192241549491882),
 (u'unrealistic', 0.712104856967926),
 (u'unnecessary', 0.7102534770965576),
 (u'comical', 0.7010959982872009),
 (u'intense', 0.7007806897163391),
 (u'contrived', 0.6941515803337097)]

In [53]:
model.most_similar("attrocious")

KeyError: "word 'attrocious' not in vocabulary"